In [71]:
# Import key librarys

import pandas as pd
import numpy as np

import plotly.express as px

import chart_studio
import chart_studio.plotly as py
import chart_studio.tools as tls

In [86]:
# Define level
level = 'National_5'
level_label = level.replace('_', ' ')

# Import data
df = pd.read_excel("./more_data/attainment-statistics-(august)-2022-provisional-main.xlsx", 
                        sheet_name = level, skiprows = 2)

df17 = pd.read_excel("./more_data/ASR2017_" + level + ".xls", 
                        sheet_name = "Table 4", skiprows = 7)

df16 = pd.read_excel("./more_data/ASR2016_" + level + ".xls", 
                        sheet_name = "Table 4", skiprows = 7)

In [87]:
def tidy_up_1(df):
    
    # Remove spaces and replace with '_'
    df.columns = df.columns.str.replace(' ','_')

    #df.drop(df[df.Grade_A_Count_2022 == '[z]'].index, inplace=True)
    df = df.replace('[z]', np.nan)
    df = df.replace('[c]', np.nan)
    df = df.replace('[low]', np.nan)

    # Remove last row
    df = df.drop(df.index[-1])

    # Remove columns with Percentage in the heading
    df = df[df.columns.drop(list(df.filter(regex='Percentage')))]
    
    return df
    
df = tidy_up_1(df)

In [88]:
def tidy_up_1a(df, year):
    
    # Sort out column titles
    df = df.rename({'TITLE': 'Subject'}, axis=1)
    df = df.rename({'ENTRIES': 'Entries_' + year}, axis=1)
    
    df = df.rename({'A': 'As_' + year}, axis=1)
    df = df.rename({'B': 'Bs_' + year}, axis=1)
    df = df.rename({'C': 'Cs_' + year}, axis=1)
    df = df.rename({'D': 'Ds_' + year}, axis=1)
    
    #df = df.rename({'NAs' + year: 'NAs_' + year}, axis=1)

    # Remove last row
    df = df.drop(df.index[-1])
 
    return df
    
df16 = tidy_up_1a(df17, '2016')
df17 = tidy_up_1a(df17, '2017')

In [89]:
def count(df, year):

    # Calculate number of Bs, Cs and Ds
    df['As_' + year] = round(df['Entries_' + year] * df['As_' + year] / 100)
    df['Bs_' + year] = round(df['Entries_' + year] * df['Bs_' + year] / 100)
    df['Cs_' + year] = round(df['Entries_' + year] * df['Cs_' + year] / 100)
    df['Ds_' + year] = round(df['Entries_' + year] * df['Ds_' + year] / 100)
    
    # Calculate number of NAs
    df['NAs_' + year] = round(df['Entries_' + year] 
                              - df['As_' + year]
                              - df['Bs_' + year]
                              - df['Cs_' + year]
                              - df['Ds_' + year])
        
    return df

df16 = count(df16, '2016')
df17 = count(df17, '2017')

In [90]:
df16.tail()

,Subject,Entries_2016,As_2016,Bs_2016,Cs_2016,Ds_2016,NAs_2016
44,Psychology,622.0,226.0,120.0,92.0,39.0,145.0
45,"Religious, Moral and Philosophical Studies",2623.0,819.0,482.0,490.0,201.0,631.0
46,Sociology,241.0,97.0,51.0,38.0,14.0,41.0
47,Spanish,4488.0,2642.0,829.0,558.0,184.0,275.0
48,Urdu,74.0,50.0,11.0,7.0,3.0,3.0


In [91]:
def tidy_up_2(df):

    # Create a list of years
    years = ['2022', '2021', '2020', '2019', '2018']

    # Sort out column titles
    for year in years:
        df = df.rename({'Grade_A_Count_' + year: 'As_' + year}, axis=1)
        df = df.rename({'Grades_A-B_Count_' + year: 'A-Bs_' + year}, axis=1)
        df = df.rename({'Grades_A-C_Count_' + year: 'A-Cs_' + year}, axis=1)
        df = df.rename({'Grades_A-D_Count_' + year: 'A-Ds_' + year}, axis=1)
        df = df.rename({'No_Award_Count_' + year: 'NAs_' + year}, axis=1)

    # Calculate number of Bs, Cs and Ds
    for year in years:
        df['Bs_' + year] = df['A-Bs_' + year] - df['As_' + year]
        df['Cs_' + year] = df['A-Cs_' + year] - df['A-Bs_' + year]
        df['Ds_' + year] = df['A-Ds_' + year] - df['A-Cs_' + year]

    # Remove original columns
    for year in years:
        df.drop('A-Bs_' + year, axis=1, inplace=True)
        df.drop('A-Cs_' + year, axis=1, inplace=True)
        df.drop('A-Ds_' + year, axis=1, inplace=True)
        
    return df
        
df = tidy_up_2(df)

In [92]:
# Sort out subjects names in 2016 and 2017 datasets

def tidy_up_2a(df):

    # Update names
    di = {'Hospitality: Practical Cake Craft': 'Practical Cake Craft',
          'Hospitality: Practical Cookery': 'Practical Cookery',
          'Cantonese': 'Chinese Languages', 'Gàidhlig': 'Gaidhlig'}
    
    df = df.replace({"Subject": di})
    
    # Remove duplicate names
    values = ['Cruinn-eolas (Geography)', 'Nuadh-Eolas (Modern Studies)', 'Eachdraidh (History)', 
        'Matamataig (Mathematics)', 'Matamataig Fad-bheatha (Lifeskills Mathematics)',
            'Mandarin (Simplified)', 'Mandarin (Traditional)']

    df = df.loc[~df['Subject'].isin(values)]
    
    return df

df16 = tidy_up_2a(df16)
df17 = tidy_up_2a(df17)

In [93]:
df.head()

,Subject,As_2022,NAs_2022,Entries_2022,As_2021,NAs_2021,Entries_2021,As_2020,NAs_2020,Entries_2020,...,Ds_2021,Bs_2020,Cs_2020,Ds_2020,Bs_2019,Cs_2019,Ds_2019,Bs_2018,Cs_2018,Ds_2018
0,Accounting,440,85.0,855,510,50.0,900,515,30.0,910,...,60,190,125,50,180,135,65,145,130,100
1,Administration and IT,1680,305.0,5410,2665,110.0,5420,2000,100.0,5190,...,295,1530,1255,305,1310,1085,645,1345,1020,590
2,Applications of Mathematics,3570,2485.0,14305,2440,2055.0,10750,3020,1285.0,10430,...,1815,2060,2770,1295,720,825,830,485,490,485
3,Art and Design,4480,190.0,10095,4750,160.0,10020,4570,100.0,9915,...,450,3045,1940,260,2695,2180,930,2530,1720,795
4,Biology,7350,2240.0,22575,7745,1980.0,21195,7690,1135.0,21785,...,3145,5030,5440,2495,4455,4390,3925,4535,4215,3475


In [94]:
# Concat together three dfs (on Subject as index)

dfc = pd.concat([df.set_index('Subject'), df17.set_index('Subject'), df16.set_index('Subject')], 
                  axis=1).reset_index()

In [95]:
def as_per(df):

    # Create a list of years
    years = ['2022', '2021', '2020', '2019', '2018', '2017', '2016']

    # Calculate perc of As
    for year in years:
        
        df['perAs_' + year] = round(df['As_' + year] / df['Entries_' + year], 2)
        
    return df

dfc = as_per(dfc)
        
dfc.tail()

,Subject,As_2022,NAs_2022,Entries_2022,As_2021,NAs_2021,Entries_2021,As_2020,NAs_2020,Entries_2020,...,Cs_2016,Ds_2016,NAs_2016,perAs_2022,perAs_2021,perAs_2020,perAs_2019,perAs_2018,perAs_2017,perAs_2016
45,Sociology,85.0,40.0,280.0,120.0,15.0,270.0,95.0,15.0,285.0,...,38.0,14.0,41.0,0.30,0.44,0.33,0.29,0.27,0.40,0.40
46,Spanish,3410.0,195.0,5765.0,3200.0,130.0,5425.0,3275.0,60.0,5815.0,...,558.0,184.0,275.0,0.59,0.59,0.56,0.53,0.58,0.59,0.59
47,Urdu,55.0,NaN,70.0,50.0,0.0,65.0,80.0,0.0,85.0,...,7.0,3.0,3.0,0.79,0.77,0.94,0.87,0.78,0.68,0.68
48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,Lifeskills Mathematics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,486.0,295.0,1096.0,NaN,NaN,NaN,NaN,NaN,0.13,0.13


In [96]:
# Convert into long format

def tolong(df):
    ldf = pd.melt(df, id_vars = ['Subject'], var_name = 'abc', value_name ='Count')
    ldf['Grades'] = ldf['abc'].apply(lambda x: (x.rsplit('_', 1))[0])
    ldf['Year'] = ldf.pop('abc').apply(lambda x: (x.split('_'))[-1])
    
    return ldf
    
ldf = tolong(dfc)

ldf.tail()

,Subject,Count,Grades,Year
2445,Sociology,0.40,perAs,2016
2446,Spanish,0.59,perAs,2016
2447,Urdu,0.68,perAs,2016
2448,NaN,NaN,perAs,2016
2449,Lifeskills Mathematics,0.13,perAs,2016


In [97]:
ldf = ldf[ldf.Grades == 'perAs']
ldf.dropna(inplace=True)

ldf.tail()

,Subject,Count,Grades,Year
2444,"Religious, Moral and Philosophical Studies",0.31,perAs,2016
2445,Sociology,0.40,perAs,2016
2446,Spanish,0.59,perAs,2016
2447,Urdu,0.68,perAs,2016
2449,Lifeskills Mathematics,0.13,perAs,2016


In [98]:
ldf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336 entries, 2100 to 2449
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Subject  336 non-null    object 
 1   Count    336 non-null    float64
 2   Grades   336 non-null    object 
 3   Year     336 non-null    object 
dtypes: float64(1), object(3)
memory usage: 13.1+ KB


In [99]:
# Plot graph

# Create a list of subjects in alphabetical order
subjects = sorted(ldf['Subject'].unique())
years = sorted(ldf['Year'].unique())

fig = px.line(ldf, x = "Year", y = "Count", 
                color = "Subject",
               # facet_row = 'School',
                width = 800,
                height = 1150,
                category_orders = {'Subject' : subjects, 'Year' : years},
                labels = {"Count": "Percentage"},
                title = 'Percentage of As per entry per subject from 2016 - 2022 at ' + level_label
                )

fig.update_traces(mode = "markers+lines")

fig.update_yaxes(matches=None, rangemode = "tozero")

fig.write_html('./graphs/As_percentage_line_' + level + '.html')

fig.show()

In [101]:
# Setup Chart Studio
username = '#####' # your username
api_key = '#####' # your api key - go to profile > settings > regenerate key
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

# Upload to Chart Studio
py.plot(fig, filename = 'As_percentage_line_2016_' + level, auto_open=True)

PlotlyRequestError: No such user